## PROJECT. Исследование поведения пользователей (SF_38.9)
### Описание гипотезы
#### Проверим два предположения:
1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
2. Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?
Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2018 году.



### Ход проверки

Часть 1. Преобразование типов

In [4]:
import pandas as pd
df = pd.read_csv('7_4_Events.csv')
df2 = pd.read_csv('purchase.csv')
cond = (df.start_time>='2018-01-01') & (df.start_time<'2019-01-01') & (df.event_type=='registration')
cond2 = (df2.event_datetime>='2018-01-01') & (df2.event_datetime<'2019-01-01') & (df.event_type=='registration')
registered = df[cond]['user_id'].to_list()
registered2 = df2[cond2]['user_id'].to_list()
events = df[df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')
df2['event_type'] = 'purchase'
purchase = df2[df2.user_id.isin(registered2)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')
print(events.start_time)
print(purchase.event_datetime)

C:\Users\user\AppData\Local\Temp/ipykernel_13384/2141845645.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  registered2 = df2[cond2]['user_id'].to_list()


51405    2018-01-01 03:48:40
51406    2018-01-01 04:07:25
51407    2018-01-01 08:35:10
51408    2018-01-01 11:54:47
51409    2018-01-01 13:28:07
                 ...        
118359   2019-01-01 03:26:18
118360   2019-01-01 03:29:25
118362   2019-01-01 05:04:52
118363   2019-01-01 05:42:11
118364   2019-01-01 05:50:36
Name: start_time, Length: 66959, dtype: datetime64[ns]
1164   2018-01-01 12:37:02
1166   2018-01-01 23:23:49
1167   2018-01-02 09:53:55
1168   2018-01-02 18:48:55
1170   2018-01-02 23:28:43
               ...        
2732   2018-12-24 19:33:02
2737   2018-12-26 00:06:05
2738   2018-12-26 09:53:20
2746   2018-12-27 21:18:31
2752   2018-12-29 18:35:39
Name: event_datetime, Length: 510, dtype: datetime64[ns]


Часть 2. Сделаем объединенный датафрейм из событий и оплат

In [5]:
import pandas as pd
df = pd.read_csv('7_4_Events.csv')
df2 = pd.read_csv('purchase.csv')
total_events_df = pd.concat([df,df2],sort=False)
print(total_events_df)

         id       event_type selected_level           start_time  tutorial_id  \
0     28903     registration            NaN  2016-05-11T23:40:55          NaN   
1     28904     registration            NaN  2016-05-11T23:49:58          NaN   
2     28905     registration            NaN  2016-05-12T00:53:07          NaN   
3     28906   tutorial_start            NaN  2016-05-12T01:32:20      17562.0   
4     28907  tutorial_finish            NaN  2016-05-12T01:34:53      17562.0   
...     ...              ...            ...                  ...          ...   
5951  21625              NaN            NaN                  NaN          NaN   
5952  21626              NaN            NaN                  NaN          NaN   
5953  21627              NaN            NaN                  NaN          NaN   
5954  21628              NaN            NaN                  NaN          NaN   
5955  21629              NaN            NaN                  NaN          NaN   

      user_id       event_d

Часть 3 Находим группы пользователей по уровню сложности, который был выбран для тренировок

In [ ]:
import pandas as pd
df = pd.read_csv('events.csv')
df2 = pd.read_csv('purchase.csv')
total_events_df = pd.concat([df,df2],sort=False)
print(total_events_df['selected_level'].unique())

[nan 'hard' 'easy' 'medium']


Часть 4 Сформируем группы

In [6]:
import pandas as pd
df = pd.read_csv('7_4_Events.csv')
df2 = pd.read_csv('purchase.csv')
cond = (df.start_time>='2018-01-01') & (df.start_time<'2019-01-01') & (df.event_type=='registration')
cond2 = (df2.event_datetime>='2018-01-01') & (df2.event_datetime<'2019-01-01')
registered = df[cond]['user_id'].to_list()
events = df[df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')
df2['event_type'] = 'purchase'
purchase = df2[df2.user_id.isin(registered)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')
total_events_df = pd.concat([events,purchase],sort=False)
users_with_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']
users_with_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']
users_with_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']
user_groups = [
    {'easy': users_with_easy_level},
    {'medium': users_with_medium_level},
    {'hard': users_with_hard_level}]
for group in user_groups:
    level = list(group.keys())[0]
    group_users = group[level]
    count_of_users_in_group = len(group_users)
    purchase_df_slice = purchase[purchase['user_id'].isin(group_users['user_id'])]
    percent_of_purchase = purchase_df_slice['user_id'].nunique()/count_of_users_in_group
    print ('Процент оплативших пользователей, выбравших уровень сложности {}: {:.2%}'.format(level,percent_of_purchase))
    print ()
    level_choice_df = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(group_users['user_id']))]
    if (level_choice_df['user_id'].value_counts().mean()) == [1]:
        level_choice_df = level_choice_df[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
        #Исправила
        purchase_df_slice_2 = purchase_df_slice[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
        merged_df = purchase_df_slice_2.merge(level_choice_df,on='user_id',how='inner')
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['level_choice_time']
        mean_time = merged_df['timedelta'].mean()
        print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))
        print ('Характеристики времени:')
        print (merged_df['timedelta'].describe())
    else:
        print ('Более 1 события выбора уровня сложности')

C:\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798941
Характеристики времени:
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118679
Характеристики времени:
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%     

Часть 5 Выводы

Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%. Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798941

Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%. 
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118679

Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%. Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности hard: 3 days 07:20:41.420814479

Данные результаты подтверждают предположения:
* Вероятность оплаты зависит от выбранного пользователем уровня сложности бесплатных тренировок.
* Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой.

